In [ ]:
import json
import pickle
import numpy as np
import pandas as pd
import random
random.seed(30)
import copy
import csv
from utils import Utils
from operator import itemgetter
from tweet import Tweet
from video import Video
import location
from scipy.stats import pearsonr, spearmanr, ttest_ind, mannwhitneyu, kruskal, ks_2samp, spearmanr, pearsonr, sem, t
import matplotlib.pyplot as plt
import os

## [abo | gun | blm]
campaign = 'abo'

bin_size = 7
connection_type = 'followers'
year = 2018

util = Utils(campaign, bin_size, connection_type, year)

tweet_obj = Tweet(util)
video_obj = Video(util)


In [ ]:
all_tweets_from_filtered_videos = pickle.load(open(tweet_obj.tweets_path, 'rb'))
all_users_from_filtered_videos = pickle.load(open(tweet_obj.users_path, 'rb'))
ea_tweets = tweet_obj.getAvailableTweets(0.2)

# Cross-cutting Exposure

In [ ]:
user_leanings_scores = pickle.load(open(tweet_obj.ea_users_inferred_leanings_scores_path, 'rb'))
tweets = ea_tweets
video_leanings_probs = tweet_obj.assignVideoLeaningLabels(ea_tweets)
print(len(tweets.keys()))

vids = tweet_obj.separateVideosByLeaning(video_leanings_probs)

filtered_video_ids = tweet_obj.getFilteredVideoIds()

c_1_k = {}
c_2_k = {}
for party in ["L", "R", "N"]:
    c_1_k[party] = {'original': 0, 'retweet': 0, 'quoted': 0, 'reply': 0, 'total': 0, 'users': []}
    c_2_k[party] = {'original': 0, 'retweet': 0, 'quoted': 0, 'reply': 0, 'total': 0, 'users': []}

for tid in tweets:
    uid = tweets[tid]['_source']['user_id_str']
    user_party = None
    if (user_leanings_scores[uid]['left'] + user_leanings_scores[uid]['right']) != 0:
        right_prob = float(user_leanings_scores[uid]['right']) / (user_leanings_scores[uid]['left'] + user_leanings_scores[uid]['right'])
        if right_prob > tweet_obj.predefined_video_leaning_thr['right']:
            user_party = 'R'
        elif right_prob < tweet_obj.predefined_video_leaning_thr['left']:
            user_party = 'L'
        else:
            user_party = 'N'
         
    if user_party != None:
        tweet_type = None
        retweeted_tweet_id_str = tweets[tid]['_source']['retweeted_tweet_id_str']
        quoted_tweet_id_str = tweets[tid]['_source']['quoted_tweet_id_str']
        reply_user_id_str = tweets[tid]['_source']['reply_user_id_str']
        original_video_ids = tweets[tid]['_source']['original_vids'].split(';')
        retweeted_video_ids = tweets[tid]['_source']['retweeted_vids'].split(';')
        quoted_video_ids = tweets[tid]['_source']['quoted_vids'].split(';')
        video_ids = list(set(original_video_ids + retweeted_video_ids + quoted_video_ids))
        if 'N' in video_ids:
            video_ids.remove('N')

        if retweeted_tweet_id_str != None and retweeted_tweet_id_str != 'N':
            tweet_type = 'retweet'
        elif (retweeted_tweet_id_str == None or retweeted_tweet_id_str == 'N') and (quoted_tweet_id_str != None and quoted_tweet_id_str != 'N'):
            tweet_type = 'quoted'
        elif reply_user_id_str != None and reply_user_id_str != 'N':
            tweet_type = 'reply'
        else:
            tweet_type = 'original'

        for vid in video_ids:
            if vid in filtered_video_ids:
                if vid in vids['L']:
                    c_1_k[user_party][tweet_type] += 1
                    c_1_k[user_party]['total'] += 1
                    c_1_k[user_party]['users'].append(uid)
                elif vid in vids['R']:
                    c_2_k[user_party][tweet_type] += 1
                    c_2_k[user_party]['total'] += 1
                    c_2_k[user_party]['users'].append(uid)


pro_left_original =  float(c_1_k['L']['original'])/sum([c_1_k[p]['original'] for p in c_1_k])
pro_left_retweet =   float(c_1_k['L']['retweet'])/sum([c_1_k[p]['retweet'] for p in c_1_k])
pro_left_quoted =    float(c_1_k['L']['quoted'])/sum([c_1_k[p]['quoted'] for p in c_1_k])
pro_left_reply =     float(c_1_k['L']['reply'])/sum([c_1_k[p]['reply'] for p in c_1_k])
pro_left_total =     float(c_1_k['L']['total'])/sum([c_1_k[p]['total'] for p in c_1_k])
pro_left_users =     float(len(set(c_1_k['L']['users'])))/sum([len(set(c_1_k[p]['users'])) for p in c_1_k])
pro_right_original = float(c_1_k['R']['original'])/sum([c_1_k[p]['original'] for p in c_1_k])
pro_right_retweet =  float(c_1_k['R']['retweet'])/sum([c_1_k[p]['retweet'] for p in c_1_k])
pro_right_quoted =   float(c_1_k['R']['quoted'])/sum([c_1_k[p]['quoted'] for p in c_1_k])
pro_right_reply =    float(c_1_k['R']['reply'])/sum([c_1_k[p]['reply'] for p in c_1_k])
pro_right_total =    float(c_1_k['R']['total'])/sum([c_1_k[p]['total'] for p in c_1_k])
pro_right_users =    float(len(set(c_1_k['R']['users'])))/sum([len(set(c_1_k[p]['users'])) for p in c_1_k])

anti_left_original =  float(c_2_k['L']['original'])/sum([c_2_k[p]['original'] for p in c_2_k])
anti_left_retweet =   float(c_2_k['L']['retweet'])/sum([c_2_k[p]['retweet'] for p in c_2_k])
anti_left_quoted =    float(c_2_k['L']['quoted'])/sum([c_2_k[p]['quoted'] for p in c_2_k])
anti_left_reply =     float(c_2_k['L']['reply'])/sum([c_2_k[p]['reply'] for p in c_2_k])
anti_left_total =     float(c_2_k['L']['total'])/sum([c_2_k[p]['total'] for p in c_2_k])
anti_left_users =     float(len(set(c_2_k['L']['users'])))/sum([len(set(c_2_k[p]['users'])) for p in c_2_k])
anti_right_original = float(c_2_k['R']['original'])/sum([c_2_k[p]['original'] for p in c_2_k])
anti_right_retweet =  float(c_2_k['R']['retweet'])/sum([c_2_k[p]['retweet'] for p in c_2_k])
anti_right_quoted =   float(c_2_k['R']['quoted'])/sum([c_2_k[p]['quoted'] for p in c_2_k])
anti_right_reply =    float(c_2_k['R']['reply'])/sum([c_2_k[p]['reply'] for p in c_2_k])
anti_right_total =    float(c_2_k['R']['total'])/sum([c_2_k[p]['total'] for p in c_2_k])
anti_right_users =    float(len(set(c_2_k['R']['users'])))/sum([len(set(c_2_k[p]['users'])) for p in c_2_k])

print(pro_right_original, pro_right_retweet, pro_right_quoted, pro_right_reply, pro_right_total, pro_right_users)
print(anti_left_original, anti_left_retweet, anti_left_quoted, anti_left_reply, anti_left_total, anti_left_users)


# Find example videos and networks based on nw_closeness_centrality_gini

In [ ]:
# read structural measures
structural_measures = pickle.load(open(tweet_obj.ea_network_structural_measures_path, 'rb'))
video_leanings_probs = tweet_obj.assignVideoLeaningLabels(ea_tweets)
tmp_video_leanings_probs = {}
for vid in video_leanings_probs:
    tmp_video_leanings_probs[vid] = video_leanings_probs[vid]['right']

sorted_video_leanings_probs = sorted(tmp_video_leanings_probs.items(), key=itemgetter(1))
for pair in sorted_video_leanings_probs:
    print(pair, structural_measures[pair[0]]['nw_closeness_centrality_gini'], structural_measures[pair[0]]['nw_size'])

sub_followings_dict = pickle.load(open(tweet_obj.ea_users_sub_followings_path, 'rb'))

In [ ]:
## Create network graph and visualize it
import networkx as nx
import igraph as ig

#vid = 'ewLkgfpH6fk'
vid = '5CoxmjwFn2Q'

uids = []
for tid in ea_tweets:
    uid = ea_tweets[tid]['_source']['user_id_str']
    original_video_ids = ea_tweets[tid]['_source']['original_vids'].split(';')
    retweeted_video_ids = ea_tweets[tid]['_source']['retweeted_vids'].split(';')
    quoted_video_ids = ea_tweets[tid]['_source']['quoted_vids'].split(';')
    video_ids = list(set(original_video_ids + retweeted_video_ids + quoted_video_ids))
    if 'N' in video_ids:
        video_ids.remove('N')
    if vid in video_ids:
        uids.append(uid)
uids = list(set(uids))
print('len(uids):', len(uids))

## create graph
G = ig.Graph(directed=True)
G.add_vertices(len(uids))
for uid in uids:
    for fid in sub_followings_dict[uid]:
        if fid in uids and fid != uid:
            G.add_edges([(uids.index(uid), uids.index(fid))])
G.degree(mode="out")

print('G:', G.vcount(), G.ecount())

A = G.get_edgelist()
G2 = nx.DiGraph(A) # In case your graph is directed
for uid in uids:
    if uids.index(uid) not in G2.nodes:
        G2.add_node(uids.index(uid))
closeness_centralities = nx.algorithms.closeness_centrality(G2, wf_improved=True)
nw_closeness_centrality_gini = tweet_obj.util.gini(list(closeness_centralities.values()))
print('G2:', G2.number_of_nodes(), G2.number_of_edges())
print(nw_closeness_centrality_gini)


# visualize the network
for nid in range(len(uids)):
    G.vs[nid]["size"] = 100
    G.vs[nid]["color"] = 'white'
    G.vs[nid]["label"] = "U{}".format(nid)
    G.vs[nid]["label_size"] = 35

visual_style = {}
visual_style["layout"] = G.layout_star(center=27)
visual_style["edge_width"] = 2
visual_style["bbox"] = (3500, 3500)
ig.plot(G, "deneme_{}.pdf".format(vid), **visual_style)

'''
layout = []

x = -1
y = 0
for i in range(len(uids)):
    layout.append((x, y))
    x += 0.05

visual_style = {}
visual_style["bbox"] = (5000, 1000)
visual_style["layout"] = layout
visual_style["edge_curved"] = 0.3
visual_style["vertex_label_size"] = 24
visual_style["margin"] = 250
#visual_style["edge_width"] = [float(width)/100 for width in edge_weights]
ig.plot(G, "deneme_lll.pdf", **visual_style)
'''

# Sample videos based on their score percentile for sanity check of leaning assignment

In [ ]:
video_leanings_probs = tweet_obj.assignVideoLeaningLabels(ea_tweets)
vids = list(video_leanings_probs.keys())
scores = [video_leanings_probs[vid]['right'] for vid in vids]

def sampleVideosByScore(scores):
    cuts = np.linspace(0, 100, 11)
    results = []
    for i in range(len(cuts)-1):
        results.append([])
        lower = np.percentile(scores, cuts[i])
        upper = np.percentile(scores, cuts[i+1])
        perc_scores_inds = np.where((scores>=lower) & (scores<upper))[0]
        rndm_perc_scores_inds = random.sample(list(perc_scores_inds), k=5)
        for ind in rndm_perc_scores_inds:
            results[i].append(vids[ind])
    return results

result = sampleVideosByScore(scores)

ind = 8
cuts = np.linspace(0, 100, 11)
print(cuts)
#print(cuts[ind], cuts[ind+1])
print(np.percentile(scores, cuts[ind]), np.percentile(scores, cuts[ind+1]))
for vid in result[ind]:
    print(vid, video_leanings_probs[vid]['right'])

aa = []
for i in range(len(result)):
    for j in range(len(result[i])):
        aa.append(result[i][j])

print(len(aa), len(set(aa)))
random.shuffle(aa)

for vid in aa:
    print(vid, 'https://www.youtube.com/watch?v='+vid, video_leanings_probs[vid]['right'])